<a href="https://colab.research.google.com/github/amitaipat-create/Session2/blob/main/Copy_of_Mission_2_Market_Research_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Market Research Agent

In [ ]:
%pip install langchain==0.3.27 langchain-openai==0.3.29 langchain-community==0.3.27 wikipedia tavily-python pydantic==2.11.10 python-dotenv==1.0.1 requests --quiet

In [ ]:
import os
import getpass
import json
from dotenv import load_dotenv
from typing import Optional, Dict, Any
import requests

# Load environment variables from .env file (for local development)
load_dotenv()

In [ ]:
# OpenAI API Key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
# Tavily API Key (optional - for web search)
if not os.environ.get("TAVILY_API_KEY"):
    tavily_key = getpass.getpass("Enter your Tavily API key (or press Enter to skip): ")
    if tavily_key:
        os.environ["TAVILY_API_KEY"] = tavily_key

In [ ]:
# Alpha Vantage API Key (optional - for stock/market data)
if not os.environ.get("ALPHA_VANTAGE_API_KEY"):
    alpha_key = getpass.getpass("Enter your Alpha Vantage API key (or press Enter to skip): ")
    if alpha_key:
        os.environ["ALPHA_VANTAGE_API_KEY"] = alpha_key

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_community.tools import WikipediaQueryRun, TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper

# For Colab/Jupyter display (optional)
try:
    from IPython.display import display, Markdown
except ImportError:
    def display(x): print(x)
    def Markdown(x): return x

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

In [ ]:
# ============================================================================
# TOOLS: Building the Agent's Capabilities
# ============================================================================

# Tool 1: Wikipedia search for background information
wikipedia = WikipediaAPIWrapper()
search_wikipedia = WikipediaQueryRun(api_wrapper=wikipedia)

In [ ]:
# Tool 2: Tavily web search for current information
# Only add if API key is available
tavily_search = None
if os.environ.get("TAVILY_API_KEY"):
    tavily_search = TavilySearchResults(max_results=3)

In [ ]:
# Tool 3: Alpha Vantage API for stock/market data
@tool
def get_stock_data(symbol: str, data_type: str = "overview") -> str:
    """Get stock or company data from Alpha Vantage API

    Args:
        symbol: Stock symbol (e.g., 'AAPL', 'TSLA', 'MSFT')
        data_type: Type of data - 'overview' (company info) or 'quote' (current price)

    Returns:
        Company or stock data as formatted string
    """
    api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")
    if not api_key:
        return "Error: ALPHA_VANTAGE_API_KEY not found. Get free key from https://www.alphavantage.co/support/#api-key"

    try:
        if data_type == "overview":
            url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}"
        elif data_type == "quote":
            url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={api_key}"
        else:
            return f"Error: data_type must be 'overview' or 'quote'"

        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Format the response
        if data_type == "overview":
            if "Symbol" in data:
                return f"""Company: {data.get('Name', 'N/A')} ({data.get('Symbol', 'N/A')})
Sector: {data.get('Sector', 'N/A')}
Industry: {data.get('Industry', 'N/A')}
Description: {data.get('Description', 'N/A')[:200]}...
Market Cap: {data.get('MarketCapitalization', 'N/A')}
P/E Ratio: {data.get('PERatio', 'N/A')}
52 Week High: {data.get('52WeekHigh', 'N/A')}
52 Week Low: {data.get('52WeekLow', 'N/A')}"""
            else:
                return f"Error: {data.get('Note', 'Invalid response from API')}"
        else:  # quote
            if "Global Quote" in data:
                quote = data["Global Quote"]
                return f"""Symbol: {quote.get('01. symbol', 'N/A')}
Price: ${quote.get('05. price', 'N/A')}
Change: {quote.get('09. change', 'N/A')} ({quote.get('10. change percent', 'N/A')})
Volume: {quote.get('06. volume', 'N/A')}
Latest Trading Day: {quote.get('07. latest trading day', 'N/A')}"""
            else:
                return f"Error: {data.get('Note', 'Invalid response from API')}"
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
# Collect all tools (only add tavily if key is available)
tools = [search_wikipedia, tavily_search, get_stock_data]
# if tavily_search:
#     tools.append(tavily_search)
# tools.append(get_stock_data)

In [ ]:
# ============================================================================
# AGENT SETUP: With Memory for Conversation History
# ============================================================================

# Create memory to remember conversation history
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [ ]:
# Create prompt template with memory support
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a Market Research Reporter Agent. Your job is to:
    1. Research markets, companies, or topics thoroughly
    2. Use available tools to gather information (Wikipedia, web search, APIs)
    3. Provide comprehensive, well-structured research reports
    4. Remember previous questions and maintain context in conversations

    Always use tools when you need current information or data."""),
    ("placeholder", "{chat_history}"),  # Conversation history
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),  # Agent's reasoning
])

# Create agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create executor with memory
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True  # Set to False to hide detailed execution
)

print("✓ Market Research Reporter Agent ready!")

In [ ]:
# ============================================================================
# EXAMPLE 1: Research a Company
# ============================================================================

result1 = agent_executor.invoke({
    "input": "Research Tesla and provide a summary of the company"
})
display(Markdown(f"**Research Report:**\n{result1['output']}"))

In [ ]:
# ============================================================================
# EXAMPLE 2: Research with Follow-up Questions
# ============================================================================

# First question
result2 = agent_executor.invoke({
    "input": "What is the current state of the electric vehicle market?"
})
display(Markdown(f"**Response:**\n{result2['output']}"))

In [ ]:
# Follow-up question (agent remembers the context!)
result3 = agent_executor.invoke({
    "input": "What are the main competitors in that market?"
})
display(Markdown(f"**Follow-up Response:**\n{result3['output']}"))

In [ ]:
# ============================================================================
# EXAMPLE 3: Research with Stock Data
# ============================================================================

# Only run if Alpha Vantage key is available
if os.environ.get("ALPHA_VANTAGE_API_KEY"):
    result4 = agent_executor.invoke({
        "input": "Research Apple (AAPL). Get company overview from stock API and search Wikipedia for background"
    })
    display(Markdown(f"**Research with Stock Data:**\n{result4['output']}"))
else:
    print("\nNote: Alpha Vantage API key not set. Example 3 skipped.")
    print("Get free API key from: https://www.alphavantage.co/support/#api-key")

In [ ]:
# ============================================================================
# EXAMPLE 4: Multi-Source Research
# ============================================================================

result5 = agent_executor.invoke({
    "input": "Research artificial intelligence industry. Get information from Wikipedia and search for latest 2024 developments"
})
display(Markdown(f"**Multi-Source Research:**\n{result5['output']}"))

In [ ]:
# ============================================================================
# EXERCISES
# ============================================================================

# EXERCISE 1: Research a company of your choice
# agent_executor.invoke({"input": "Research [Company Name] and provide a comprehensive report"})

# EXERCISE 2: Ask follow-up questions
# First: "What is the renewable energy market?"
# Then: "What are the growth trends?" (agent should remember the market context)

# EXERCISE 3: Combine multiple tools
# Ask: "Research [Topic] using Wikipedia for background and web search for latest news"

# EXERCISE 4: Research with stock data (if Alpha Vantage key available)
# Ask: "Research [Company] (SYMBOL) and get stock information"
# Example: "Research Microsoft (MSFT) and provide company overview and current stock price"